In [1]:
from time import sleep
from random import randint
import re
import numpy as np

In [2]:
import requests
from requests import get
from bs4 import BeautifulSoup as bs 
import lxml
import pandas as pd

In [3]:
from bs4.element import NavigableString, Tag

def extractNavigableStrings(context):
    strings = []
    for e in context.children:
        if isinstance(e, NavigableString):
            strings.append(e)
        if isinstance(e, Tag):
            strings.extend(extractNavigableStrings(e))
    return strings




def get_player(soup):
    
    card = soup.find_all("div" ,attrs = {"class" :"card-body"})
    span = card[1].find_all("span")
    
    name_rating = soup.find('h5')
    list_name_rating = extractNavigableStrings(name_rating)
    
    
    #break down span in to varibels 
    name =list_name_rating[0]
    rating1 = list_name_rating[1]
    rating2 = list_name_rating[3]
    
    
    #teams
    
    teams = soup.find_all("a" , attrs={"class":"link-team"})
    try:
        nat_team = teams[1].text
        team = teams[3].text
    except :
        team = teams[1].text
        nat_team = np.nan
        
    #nation
    
    nation = soup.find("a", attrs={"class" :"link-nation"})
    nat = nation['title']
    
    
    height = span[1].text
    weight = span[4].text
    preferred_foot = span[6].text
    birth_date = span[7].text
    age = span[8].text
    positions = span[9].text
    
    work_rate_list = ['High' ,'Medium' , 'Low']
    
    for rate in work_rate_list:
    
        if soup.find('span' , string = re.compile(rate)) != None:
            work_rate = soup.find('span' , string = re.compile(rate)).text
    
    try:
        value = soup.find_all("p" , attrs={"class":"data-currency data-currency-dollar"})[0].text
        wage = soup.find_all("p" , attrs={"class":"data-currency data-currency-dollar"})[1].text
    except IndexError :
        value = np.nan
        wage = np.nan
        print(f'eror:{name} ')
    
    fet_list = [name, rating1,rating2,nat_team, team, nat,height, weight, preferred_foot, birth_date, age, positions, work_rate, value, wage ]
    
    return fet_list
    






#function to get each link of player on current page 
def get_links(soup):
    
    tag_links = soup.find_all("td" , attrs={"data-title":"Name"})
    links = ['https://www.fifaindex.com/' + link.find("a")['href'] for link in tag_links]
    
    return links 

In [9]:
df = pd.DataFrame(columns=['name', 'rating1','rating2','nat_team','team','nat','height', 'weight', 'preferred_foot',
                           'birth_date', 'age', 'positions', 'work_rate', 'value', 'wage' ])

In [ ]:
def get_data(start, stop):
    
    
    df = pd.DataFrame(columns=['name', 'rating1','rating2','nat_team','team','nat','height', 'weight', 'preferred_foot',
                           'birth_date', 'age', 'positions', 'work_rate', 'value', 'wage' ])
    
    for i in range(start,stop):
    
        sleep(randint(1,3))
        url = "https://www.fifaindex.com/players/"
        web_page = get(url+str(i))
    
        soup = bs(web_page.content)
    
        links = get_links(soup)
    
        for link in links:
        
            little_soup = bs(get(link).content)
            df.loc[len(df)] = get_player(little_soup)
        
        print(f"page number {i}")
        
    return df
    
    
    
 



In [6]:
df = get_data(1,645)
df.to_csv('data1_654')

In [7]:
df

,name,rating1,rating2,nat_team,team,nat,height,weight,preferred_foot,birth_date,age,positions,work_rate,value,wage
0,Lionel Messi,94,94,Argentina,FC Barcelona,Argentina,170 cm,72 kg,Left,"June 24, 1987",33,RWSTCF,Medium / Low,Value $95.500.000,Wage $560.000
1,Cristiano Ronaldo,93,93,Portugal,Juventus,Portugal,187 cm,83 kg,Right,"Feb. 5, 1985",35,STLW,High / Low,Value $58.500.000,Wage $400.000
2,Neymar Jr,92,92,Brazil,Paris Saint-Germain,Brazil,175 cm,68 kg,Right,"Feb. 5, 1992",28,LWCAM,High / Medium,Value $105.500.000,Wage $290.000
3,Virgil van Dijk,91,92,Netherlands,Liverpool,Netherlands,193 cm,92 kg,Right,"July 8, 1991",29,CB,Medium / Medium,Value $90.000.000,Wage $245.000
4,Jan Oblak,91,93,Slovenia,Atlético Madrid,Slovenia,188 cm,87 kg,Right,"Jan. 7, 1993",27,GK,Medium / Medium,Value $77.500.000,Wage $125.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19482,Tom Strannegård,48,63,NaN,AIK,Sweden,177 cm,63 kg,Left,"April 29, 2002",18,CM,Medium / Medium,Value $60.000,Wage $1.500
19483,Emil Kornvig,48,62,NaN,Lyngby BK,Denmark,170 cm,70 kg,Left,"April 28, 2000",20,RW,Medium / Medium,Value $60.000,Wage $1.500
19484,Ziyad Al Oyouni,48,70,NaN,Lincoln City,England,174 cm,69 kg,Right,"Sept. 22, 2002",18,RB,Medium / Medium,Value $60.000,Wage $1.500
19485,Ronan McKinley,48,70,NaN,Derry City,England,179 cm,74 kg,Right,"June 21, 2002",18,CM,Medium / Medium,Value $60.000,Wage $1.500
